In [3]:
import glob
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from tqdm import tqdm
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

In [5]:
df = pd.DataFrame(columns=['feature'])
labels = []
bookmark=0
emo_dict = {'angry':0, 'fearful':1, 'happy':2, 'sad':3, 'calm':4}
ravdess_files = glob.glob(os.path.join('/home/datahack/Documents/VOICE_DATASET','*','*.wav'))
for wav in tqdm(ravdess_files):
    #label_string = os.path.basename(wav).split('.')[0].split('_')[1]
    label_string = wav.split("/")[-2]
    label = emo_dict[label_string]
    
    try:
        #X, sample_rate = librosa.load(wav, res_type='kaiser_best', duration=2.5, offset=0.5, sr=44100)
        X, sample_rate = librosa.load(wav, res_type='kaiser_best', sr=48000)
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0) 
        
        labels.append(label)
        
        feature = mfccs
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1

        
    except ValueError:
        continue
    

100%|██████████| 305/305 [00:07<00:00, 41.01it/s]


In [6]:
df3 = pd.DataFrame(df['feature'].values.tolist())

In [7]:
df3['label']=np.array(labels)

In [9]:
rnewdf=df3.copy(deep=True)

In [10]:
from sklearn.utils import shuffle
rnewdf = shuffle(rnewdf)


In [11]:
rnewdf[:5]

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,256,label
195,-15.581483,-15.987067,-15.822970,-15.558836,-16.059996,-16.764061,-15.877100,-15.362707,-15.304568,-14.782032,...,-16.050462,-15.900082,-15.973406,-16.053119,-16.701738,-15.512875,-14.936444,-14.457099,-16.245404,1
292,-17.428441,-16.292398,-15.429844,-16.223066,-16.042339,-15.352880,-15.063309,-15.150067,-14.745865,-14.334231,...,-15.797796,-16.264587,-16.203997,-14.924871,-14.901760,-14.586964,-15.856836,-16.761614,-19.203843,0
249,-16.378641,-15.245748,-15.157072,-16.350706,-16.658661,-15.823436,-16.789780,-15.680457,-13.938305,-14.991134,...,-16.116484,-16.487236,-16.837500,-18.136304,-17.021971,-16.166768,-16.013828,-16.251962,-19.423200,0
6,-16.016096,-15.354762,-16.085609,-15.214061,-16.437640,-16.658628,-15.860445,-15.113874,-14.485299,-15.722691,...,-11.697349,-11.754840,-11.690804,-12.066640,-12.539812,-13.988450,-13.507364,-13.365669,-14.261080,2
75,-19.531726,-22.532773,-26.552657,-27.779262,-28.669385,-26.597004,-25.620996,-24.103831,-22.268830,-24.537104,...,-25.543127,-23.894561,-23.152223,-24.809263,-25.967208,-26.517781,-25.973606,-22.007794,-21.900753,4


In [12]:
rnewdf.isnull().any(axis=1)[rnewdf.isnull().any(axis=1) == True]

Series([], dtype: bool)

In [ ]:
rnewdf.dropna(inplace=True)

In [13]:
rlabel = pd.DataFrame({'label':rnewdf.label.values})

In [14]:
rnewdf.drop(columns='label', axis=1, inplace=True)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(rnewdf, rlabel, test_size=0.33, random_state=42)

In [16]:
y_train.shape

(204, 1)

In [17]:
from keras.utils import np_utils

In [18]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [19]:
y_train.shape

(204, 5)

In [20]:
print('Pad sequences')
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

Pad sequences


In [21]:
x_traincnn.shape

(204, 257, 1)

In [22]:
model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(257,1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(5))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 257, 128)          768       
_________________________________________________________________
activation_1 (Activation)    (None, 257, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 257, 128)          82048     
_________________________________________________________________
activation_2 (Activation)    (None, 257, 128)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 257, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 32, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 32, 128)           82048     
__________

In [24]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [25]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=32, epochs=1000, validation_data=(x_testcnn, y_test))

Train on 204 samples, validate on 101 samples
Epoch 1/1000
204/204 [==============================] - 8s 40ms/step - loss: 1.6562 - acc: 0.2157 - val_loss: 1.5986 - val_acc: 0.2574
Epoch 2/1000
204/204 [==============================] - 0s 317us/step - loss: 1.6393 - acc: 0.2010 - val_loss: 1.5996 - val_acc: 0.2475
Epoch 3/1000
204/204 [==============================] - 0s 307us/step - loss: 1.6085 - acc: 0.2353 - val_loss: 1.6000 - val_acc: 0.2475
Epoch 4/1000
204/204 [==============================] - 0s 310us/step - loss: 1.6057 - acc: 0.2402 - val_loss: 1.6039 - val_acc: 0.2475
Epoch 5/1000
204/204 [==============================] - 0s 321us/step - loss: 1.5981 - acc: 0.2157 - val_loss: 1.6046 - val_acc: 0.2376
Epoch 6/1000
204/204 [==============================] - 0s 319us/step - loss: 1.6000 - acc: 0.2255 - val_loss: 1.6047 - val_acc: 0.2376
Epoch 7/1000
204/204 [==============================] - 0s 310us/step - loss: 1.5936 - acc: 0.2696 - val_loss: 1.6078 - val_acc: 0.2574
Epo

204/204 [==============================] - 0s 321us/step - loss: 1.3461 - acc: 0.4657 - val_loss: 1.4096 - val_acc: 0.3762
Epoch 60/1000
204/204 [==============================] - 0s 295us/step - loss: 1.3331 - acc: 0.4902 - val_loss: 1.4034 - val_acc: 0.3762
Epoch 61/1000
204/204 [==============================] - 0s 292us/step - loss: 1.3233 - acc: 0.5098 - val_loss: 1.4050 - val_acc: 0.3861
Epoch 62/1000
204/204 [==============================] - 0s 297us/step - loss: 1.3133 - acc: 0.4510 - val_loss: 1.3872 - val_acc: 0.4059
Epoch 63/1000
204/204 [==============================] - 0s 282us/step - loss: 1.3091 - acc: 0.4706 - val_loss: 1.3711 - val_acc: 0.4851
Epoch 64/1000
204/204 [==============================] - 0s 291us/step - loss: 1.3090 - acc: 0.4804 - val_loss: 1.3676 - val_acc: 0.4356
Epoch 65/1000
204/204 [==============================] - 0s 299us/step - loss: 1.3005 - acc: 0.5049 - val_loss: 1.3724 - val_acc: 0.3960
Epoch 66/1000
204/204 [==============================] 

204/204 [==============================] - 0s 327us/step - loss: 1.0450 - acc: 0.5980 - val_loss: 1.2209 - val_acc: 0.5050
Epoch 119/1000
204/204 [==============================] - 0s 314us/step - loss: 1.0141 - acc: 0.6127 - val_loss: 1.1495 - val_acc: 0.5644
Epoch 120/1000
204/204 [==============================] - 0s 295us/step - loss: 1.0597 - acc: 0.5931 - val_loss: 1.1488 - val_acc: 0.5842
Epoch 121/1000
204/204 [==============================] - 0s 281us/step - loss: 1.0647 - acc: 0.5539 - val_loss: 1.1512 - val_acc: 0.5644
Epoch 122/1000
204/204 [==============================] - 0s 303us/step - loss: 1.0535 - acc: 0.5735 - val_loss: 1.1475 - val_acc: 0.5644
Epoch 123/1000
204/204 [==============================] - 0s 297us/step - loss: 1.0566 - acc: 0.5588 - val_loss: 1.1468 - val_acc: 0.5743
Epoch 124/1000
204/204 [==============================] - 0s 295us/step - loss: 1.0387 - acc: 0.5980 - val_loss: 1.1440 - val_acc: 0.5743
Epoch 125/1000
204/204 [=========================

204/204 [==============================] - 0s 303us/step - loss: 0.9422 - acc: 0.6275 - val_loss: 1.0650 - val_acc: 0.5644
Epoch 178/1000
204/204 [==============================] - 0s 339us/step - loss: 0.9346 - acc: 0.6569 - val_loss: 1.0846 - val_acc: 0.6139
Epoch 179/1000
204/204 [==============================] - 0s 292us/step - loss: 0.9373 - acc: 0.6471 - val_loss: 1.0821 - val_acc: 0.6238
Epoch 180/1000
204/204 [==============================] - 0s 300us/step - loss: 0.9387 - acc: 0.6373 - val_loss: 1.0668 - val_acc: 0.5842
Epoch 181/1000
204/204 [==============================] - 0s 300us/step - loss: 0.9204 - acc: 0.6324 - val_loss: 1.0714 - val_acc: 0.6040
Epoch 182/1000
204/204 [==============================] - 0s 286us/step - loss: 0.9397 - acc: 0.6373 - val_loss: 1.1254 - val_acc: 0.6040
Epoch 183/1000
204/204 [==============================] - 0s 292us/step - loss: 0.9299 - acc: 0.6422 - val_loss: 1.1095 - val_acc: 0.6040
Epoch 184/1000
204/204 [=========================

204/204 [==============================] - 0s 279us/step - loss: 0.8283 - acc: 0.6618 - val_loss: 1.0224 - val_acc: 0.6238
Epoch 237/1000
204/204 [==============================] - 0s 290us/step - loss: 0.8116 - acc: 0.6863 - val_loss: 1.0302 - val_acc: 0.5545
Epoch 238/1000
204/204 [==============================] - 0s 324us/step - loss: 0.8153 - acc: 0.6765 - val_loss: 1.0254 - val_acc: 0.5743
Epoch 239/1000
204/204 [==============================] - 0s 300us/step - loss: 0.8511 - acc: 0.6814 - val_loss: 1.0501 - val_acc: 0.6436
Epoch 240/1000
204/204 [==============================] - 0s 296us/step - loss: 0.8444 - acc: 0.6863 - val_loss: 1.0345 - val_acc: 0.6535
Epoch 241/1000
204/204 [==============================] - 0s 318us/step - loss: 0.8002 - acc: 0.7108 - val_loss: 1.0174 - val_acc: 0.6139
Epoch 242/1000
204/204 [==============================] - 0s 291us/step - loss: 0.8529 - acc: 0.6765 - val_loss: 1.0224 - val_acc: 0.6436
Epoch 243/1000
204/204 [=========================

...

204/204 [==============================] - 0s 314us/step - loss: 0.4007 - acc: 0.8676 - val_loss: 1.1733 - val_acc: 0.6040
Epoch 709/1000
204/204 [==============================] - 0s 311us/step - loss: 0.3840 - acc: 0.8676 - val_loss: 1.1272 - val_acc: 0.5842
Epoch 710/1000
204/204 [==============================] - 0s 299us/step - loss: 0.4083 - acc: 0.8725 - val_loss: 1.1220 - val_acc: 0.6238
Epoch 711/1000
204/204 [==============================] - 0s 313us/step - loss: 0.3844 - acc: 0.8676 - val_loss: 1.1311 - val_acc: 0.6040
Epoch 712/1000
204/204 [==============================] - 0s 288us/step - loss: 0.3450 - acc: 0.9020 - val_loss: 1.1214 - val_acc: 0.6040
Epoch 713/1000
204/204 [==============================] - 0s 295us/step - loss: 0.3523 - acc: 0.8971 - val_loss: 1.1884 - val_acc: 0.6238
Epoch 714/1000
204/204 [==============================] - 0s 319us/step - loss: 0.3867 - acc: 0.8676 - val_loss: 1.1263 - val_acc: 0.6238
Epoch 715/1000
204/204 [=========================

204/204 [==============================] - 0s 304us/step - loss: 0.3284 - acc: 0.9020 - val_loss: 1.1725 - val_acc: 0.6238
Epoch 768/1000
204/204 [==============================] - 0s 300us/step - loss: 0.3263 - acc: 0.9118 - val_loss: 1.2053 - val_acc: 0.5644
Epoch 769/1000
204/204 [==============================] - 0s 289us/step - loss: 0.3071 - acc: 0.9118 - val_loss: 1.1636 - val_acc: 0.5941
Epoch 770/1000
204/204 [==============================] - 0s 295us/step - loss: 0.3130 - acc: 0.9265 - val_loss: 1.1952 - val_acc: 0.6040
Epoch 771/1000
204/204 [==============================] - 0s 281us/step - loss: 0.3649 - acc: 0.8824 - val_loss: 1.1822 - val_acc: 0.5446
Epoch 772/1000
204/204 [==============================] - 0s 278us/step - loss: 0.3247 - acc: 0.8873 - val_loss: 1.1740 - val_acc: 0.6139
Epoch 773/1000
204/204 [==============================] - 0s 299us/step - loss: 0.3118 - acc: 0.9167 - val_loss: 1.1751 - val_acc: 0.5941
Epoch 774/1000
204/204 [=========================

204/204 [==============================] - 0s 286us/step - loss: 0.2980 - acc: 0.9118 - val_loss: 1.2139 - val_acc: 0.5941
Epoch 827/1000
204/204 [==============================] - 0s 298us/step - loss: 0.3155 - acc: 0.8922 - val_loss: 1.2271 - val_acc: 0.5743
Epoch 828/1000
204/204 [==============================] - 0s 275us/step - loss: 0.2920 - acc: 0.9167 - val_loss: 1.2249 - val_acc: 0.5842
Epoch 829/1000
204/204 [==============================] - 0s 278us/step - loss: 0.3003 - acc: 0.9216 - val_loss: 1.2293 - val_acc: 0.6040
Epoch 830/1000
204/204 [==============================] - 0s 306us/step - loss: 0.3213 - acc: 0.9167 - val_loss: 1.2671 - val_acc: 0.6238
Epoch 831/1000
204/204 [==============================] - 0s 278us/step - loss: 0.2991 - acc: 0.9069 - val_loss: 1.3563 - val_acc: 0.5644
Epoch 832/1000
204/204 [==============================] - 0s 275us/step - loss: 0.2960 - acc: 0.9069 - val_loss: 1.2242 - val_acc: 0.5545
Epoch 833/1000
204/204 [=========================

204/204 [==============================] - 0s 349us/step - loss: 0.2684 - acc: 0.9265 - val_loss: 1.2519 - val_acc: 0.5941
Epoch 886/1000
204/204 [==============================] - 0s 314us/step - loss: 0.2679 - acc: 0.9314 - val_loss: 1.3863 - val_acc: 0.5941
Epoch 887/1000
204/204 [==============================] - 0s 326us/step - loss: 0.2836 - acc: 0.9216 - val_loss: 1.2692 - val_acc: 0.5842
Epoch 888/1000
204/204 [==============================] - 0s 345us/step - loss: 0.2853 - acc: 0.9167 - val_loss: 1.3092 - val_acc: 0.6040
Epoch 889/1000
204/204 [==============================] - 0s 323us/step - loss: 0.2466 - acc: 0.9265 - val_loss: 1.3671 - val_acc: 0.5842
Epoch 890/1000
204/204 [==============================] - 0s 333us/step - loss: 0.2570 - acc: 0.9216 - val_loss: 1.4978 - val_acc: 0.5743
Epoch 891/1000
204/204 [==============================] - 0s 317us/step - loss: 0.2730 - acc: 0.9216 - val_loss: 1.2676 - val_acc: 0.5842
Epoch 892/1000
204/204 [=========================

204/204 [==============================] - 0s 341us/step - loss: 0.2360 - acc: 0.9167 - val_loss: 1.3576 - val_acc: 0.5545
Epoch 945/1000
204/204 [==============================] - 0s 311us/step - loss: 0.2368 - acc: 0.9461 - val_loss: 1.3952 - val_acc: 0.5743
Epoch 946/1000
204/204 [==============================] - 0s 299us/step - loss: 0.2186 - acc: 0.9363 - val_loss: 1.3406 - val_acc: 0.5743
Epoch 947/1000
204/204 [==============================] - 0s 308us/step - loss: 0.2814 - acc: 0.9167 - val_loss: 1.3552 - val_acc: 0.6040
Epoch 948/1000
204/204 [==============================] - 0s 296us/step - loss: 0.2355 - acc: 0.9363 - val_loss: 1.3389 - val_acc: 0.5842
Epoch 949/1000
204/204 [==============================] - 0s 281us/step - loss: 0.2121 - acc: 0.9461 - val_loss: 1.5148 - val_acc: 0.5545
Epoch 950/1000
204/204 [==============================] - 0s 307us/step - loss: 0.2465 - acc: 0.9363 - val_loss: 1.3264 - val_acc: 0.5842
Epoch 951/1000
204/204 [=========================

In [ ]:
#wav_path = '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/male_fearful.wav'
#wav_path = '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/female_sad.wav'
#wav_path = '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/female_angry.wav'
#wav_path = '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/female_calm.wav'
wav_path = '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/male_angry.wav'

In [ ]:
X, sample_rate = librosa.load(wav_path, res_type='kaiser_best', duration=2.5, sr=44100)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)

In [ ]:
mfccs_test = np.expand_dims(mfccs.reshape(-1,1),0)

In [ ]:
mfccs_test.shape

In [ ]:
model.predict_classes(mfccs_test)

In [ ]:
model.predict_proba(mfccs_test)